# Persuasion Techniques in Text of Memes

## Enironment Setup

##### Disk Setup

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# For emptying trash after run
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')
#drive_service.files().emptyTrash().execute()

In [3]:
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/"
#folder_name = "/content/drive/MyDrive/persuasion_technique_detection/"

##### Imports

In [4]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [5]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil
import os

In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from datasets import concatenate_datasets
from transformers import Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding

In [7]:
AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti


In [8]:
!nvidia-smi

Wed Jan 10 14:41:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   35C    P8              26W / 350W |      5MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Login to WandB

In [9]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_multilabel_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

wandb: Currently logged in as: mahmudfami (tumnlp). Use `wandb login --relogin` to force relogin


## Pre-trained Transformer Name

In [10]:
#checkpoint = "bert-base-cased"
#checkpoint= "vinai/bertweet-base"
#checkpoint = "xlm-roberta-base"
#checkpoint = "xlnet-base-cased"
#checkpoint = "albert-base-v2"
#checkpoint = "distilroberta-base"
#checkpoint = "microsoft/deberta-v3-base"
#checkpoint = "jayanta/twitter-roberta-base-sentiment-sentiment-memes"
#checkpoint = "vinai/bertweet-large"
checkpoint = "microsoft/deberta-v3-large"


In [11]:
summary_dir_path = folder_name + "subtask1/transformer/summaries/multilabel/summary_test_multilabel_" + checkpoint.replace("/","_") + "/"

## Data Preprocessing

In [12]:
# Initialize a counter for empty label samples
empty_label_counter = [0]

def filter_every_second_empty_label(sample):
    if len(sample['labels']) == 0:
        empty_label_counter[0] += 1

        return empty_label_counter[0] % 2 != 0
    return True

In [13]:
val_path=folder_name+"data/subtask1/validation.json"
train_path = folder_name+"data/subtask1/train.json"
test_path=folder_name+"data/subtask1/dev_unlabeled.json"
train_added_ptc = folder_name+"data/ptc/ptc_added_train.json"
train_val_dev_path = folder_name+"data/subtask1/train_val_dev.json"

#dataset_memes_2024_files = {"train": train_added_ptc,"validation": val_path}
dataset_memes_2024_files = {"train": train_path,"validation": val_path}
val_files={"val":val_path}
test_files={"test":test_path}
train_val_dev_files ={"train": train_val_dev_path, "validation": val_path}

dataset_memes_2024 = load_dataset("json",data_files=dataset_memes_2024_files)
dataset_memes_2024 = load_dataset("json",data_files=train_val_dev_files)

empty_labels = 1
if empty_labels == 0:
  dataset_memes_2024 = dataset_memes_2024.filter(lambda x : len(x['labels']) != 0)
if empty_labels == 0.5:
  dataset_memes_2024 = dataset_memes_2024.filter(filter_every_second_empty_label)

dataset_test=load_dataset("json",data_files=test_files)
dataset_val=load_dataset("json",data_files=val_files)

In [14]:
dataset_test

DatasetDict({
    test: Dataset({
        features: ['text', 'id'],
        num_rows: 1000
    })
})

In [15]:
dataset_memes_2024

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'link', 'id'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['labels', 'text', 'link', 'id'],
        num_rows: 500
    })
})

In [16]:
dataset_memes_2024["train"][5]

{'labels': [],
 'text': "PUTIN'S SECRET CAMOUFLAGE ARMY",
 'link': 'https://www.facebook.com/photo/?fbid=1261268274677920&set=g.655806642139792',
 'id': '66402'}

In [17]:
techniques = [['Black-and-white Fallacy/Dictatorship', 'Loaded Language',
       'Glittering generalities (Virtue)', 'Thought-terminating cliché',
       'Whataboutism', 'Slogans', 'Causal Oversimplification', 'Smears',
       'Name calling/Labeling', 'Appeal to authority',
       'Exaggeration/Minimisation', 'Repetition', 'Flag-waving',
       'Appeal to fear/prejudice', 'Reductio ad hitlerum', 'Doubt',
       "Misrepresentation of Someone's Position (Straw Man)",
       'Obfuscation, Intentional vagueness, Confusion', 'Bandwagon',
       'Presenting Irrelevant Data (Red Herring)']]
num_labels = len(techniques[0])
num_labels

20

### Preprocess Multi-Labels

In [18]:
mlb = MultiLabelBinarizer()
mlb.fit(techniques)


MultiLabelBinarizer()

#### Tokenize

In [19]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def tokenize_function(examples, binarizer):
    encoding = tokenizer(examples["text"], truncation=True, max_length=512)
    encoding['labels'] = binarizer.transform(examples['labels']).astype(np.float32).tolist()
    return encoding

tokenized_datasets = dataset_memes_2024.map(tokenize_function, fn_kwargs={"binarizer":mlb}, batched=True)
tokenized_datasets

/home/nlp-lab-ws23/nlp_praktikum/nlp_praktikum_env/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'link', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['labels', 'text', 'link', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [20]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["id", "text", "link"]}
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8, 20]),
 'input_ids': torch.Size([8, 56]),
 'token_type_ids': torch.Size([8, 56]),
 'attention_mask': torch.Size([8, 56])}

## Training

In [21]:
def save_json(data, file_path):
    with open(file_path, 'w+') as f:
        json.dump(data, f)

def create_json_output(y_pred, y_true,binarizer):

    # Transfrom numeric labels into textual labels
    y_pred = binarizer.inverse_transform(y_pred)
    y_true = binarizer.inverse_transform(y_true)

    # Convert y_true and y_pred to the required JSON format
    pred_json = [{"id": str(idx), "labels": pred_row} for idx, pred_row in enumerate(y_pred)]
    true_json = [{"id": str(idx), "labels": true_row} for idx, true_row in enumerate(y_true)]

    # Save to json files
    predictions_file = folder_name + f"subtask1/output_multilabel/tmp/predictions.json"
    gold_labels_file= folder_name + f"subtask1/output_multilabel/tmp/gold_labels.json"
    save_json(pred_json, predictions_file)
    save_json(true_json, gold_labels_file)
    return predictions_file, gold_labels_file

In [22]:
def calculate_hierarchical_metrics(predictions_file, gold_labels_file):
    # Run the scoring script
    scorer = folder_name + "subtask1/subtask_1_2a.py"
    command = f'python3 {scorer} --gold_file_path {gold_labels_file} --pred_file_path {predictions_file}'
    try:
      result = subprocess.run(command, shell=True, check=True , stdout=subprocess.PIPE, text=True)

    except subprocess.CalledProcessError as e:
      output = e.output
      print("error:",e)
    res=result.stdout
    output = res.strip()

    # Parse the output
    parts = output.split('\t')
    f1_h = float(parts[0].split('=')[1])
    prec_h = float(parts[1].split('=')[1])
    rec_h = float(parts[2].split('=')[1])

    return f1_h, prec_h, rec_h

In [23]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, binarizer,threshold=0.5):
    y_true = labels
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(predictions.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # compute metrics
    f1= f1_score(y_true=y_true, y_pred=y_pred, average='micro')

    # Create Json Output file
    predictions_file, gold_labels_file = create_json_output(y_pred, y_true,binarizer)
    accuracy = accuracy_score(y_true, y_pred)
    f1_h, prec_h, rec_h = calculate_hierarchical_metrics(predictions_file, gold_labels_file)

    # return as dictionary
    metrics = {'f1_hierarchical': f1_h,
               'precision_hierarchical': prec_h,
               'recall_hierarchical': rec_h,
               'f1': f1,
               'accuracy': accuracy,
               }
    # empty trash of drive
    #drive_service.files().emptyTrash().execute()
    return metrics

def prepare_compute_metrics(mlb,threshold):
  def compute_metrics(p: EvalPrediction):
    nonlocal mlb
    nonlocal threshold
    preds = p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    labels=p.label_ids
    result = multi_label_metrics(
        predictions=preds,
        labels=labels,binarizer=mlb,threshold=threshold)
    return result
  return compute_metrics

In [24]:
def train():
    try:
        # Initialize a new wandb run
        wandb.init()

        # sweep agent inputs config with hyperparameters
        config = wandb.config

        # Use config to access the hyperparameters
        threshold = config.threshold
        learning_rate = config.learning_rate
        epochs = 10

        run_name = config.run_name+f"_{str(threshold)}threshold_{str(learning_rate)}learningRate"
        wandb.run.name = run_name

        training_args = TrainingArguments(
            report_to = 'wandb',                    # enable logging to W&B
            run_name = run_name+f"_{str(threshold)}threshold_{str(learning_rate)}_learningRate",          # name of the W&B run
            load_best_model_at_end = True,
            output_dir = output_folder,
            overwrite_output_dir = True,
            save_strategy = 'steps',
            evaluation_strategy = 'steps',
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            learning_rate = learning_rate,
            num_train_epochs = epochs,
            save_steps = 1 / (epochs*2),
            eval_steps = 1 / (epochs*2),
            logging_steps = 100,
            save_total_limit=1,
            metric_for_best_model = 'f1_hierarchical',
            #gradient_accumulation_steps=4
        )

        trainer = Trainer(
            model,
            training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["validation"],
            data_collator=data_collator,
            tokenizer=tokenizer,
            compute_metrics=prepare_compute_metrics(mlb,threshold)
        )

        trainer.train()

        wandb.finish()
    except Exception as e:
        print(f"Error in training: {str(e)}")
    # empty trash of drive
    #drive_service.files().emptyTrash().execute()

In [25]:
# Search for float value before 'threshold'
def extract_threshold(string):
    match = re.search(r'(\d+\.\d+)threshold', string)
    if match:
        return float(match.group(1))
    else:
        return None
def extract_learning_rate(string):
    match = re.search(r'(\d+\.\d+)learningRate', string)
    if match:
        return float(match.group(1))
    else:
        return None

In [ ]:
run_name=f'{checkpoint}-multilabel-memes'.replace("/","_")
sweep_name=f'sweep_{run_name}'


output_folder = folder_name + f"subtask1/output_multilabel"
if not os.path.exists(output_folder+"/tmp"):
  os.makedirs(output_folder+"/tmp")

batch_size=2

# Set hyperparams in sweep configurations
sweep_config = {
    'method': 'grid',  # can be grid, random, or bayes
    'name' : sweep_name,
    'metric': {
      'name': 'eval/f1_hierarchical',
      'goal': 'maximize'
    },
    'parameters': {
        'learning_rate' : {
            'values': [5e-5]#[5e-4, 5e-5, 5e-6]
        },
        'threshold': {
            'values': [0.25, 0.3, 0.35, 0.4]#, 0.25, 0.3, 0.35, 0.4]
        },
        'run_name': {
            'value' : run_name
        }
    }
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(techniques[0]), problem_type="multi_label_classification", ignore_mismatched_sizes=True)

model.cuda()
# Start sweeps with specific configuration
sweep_id = wandb.sweep(sweep_config, project="subtask1_multilabel_transformer_encoder_classification")
wandb.agent(sweep_id, train)

# Get best model of sweep
api = wandb.Api()
sweep = api.sweep(f"subtask1_multilabel_transformer_encoder_classification/{sweep_id}")
best_run = sweep.best_run()

artifacts = best_run.logged_artifacts()

model_artifact = None
for artifact in artifacts:
    if 'model' in artifact.type:  # Adjust the condition based on your setup
        model_artifact = artifact
        break

# best model
if model_artifact != None:
  best_model = model_artifact.name
  print(f"Best Model: {best_model}")
else:
  warnings.warn(f"No model was found")

# best threshold
best_threshold = extract_threshold(best_model)
best_learning_rate = extract_learning_rate(best_model)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create sweep with ID: w9cb3h9j
Sweep URL: https://wandb.ai/tumnlp/subtask1_multilabel_transformer_encoder_classification/sweeps/w9cb3h9j


wandb: Agent Starting Run: jtbgjstk with config:
wandb: 	learning_rate: 5e-05
wandb: 	run_name: microsoft_deberta-v3-large-multilabel-memes
wandb: 	threshold: 0.25


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'run_name' was locked by 'sweep' (ignored update).


Step,Training Loss,Validation Loss,F1 Hierarchical,Precision Hierarchical,Recall Hierarchical,F1,Accuracy
438,0.236300,0.231005,0.517530,0.623550,0.442320,0.390525,0.164000
876,0.233200,0.252535,0.470240,0.549000,0.411240,0.212099,0.100000
1314,0.254600,0.250263,0.501590,0.474000,0.532580,0.301251,0.026000
1752,0.249000,0.250846,0.470240,0.549000,0.411240,0.212099,0.100000
2190,0.236700,0.250490,0.470240,0.549000,0.411240,0.212099,0.100000
2628,0.251500,0.250463,0.353000,0.490670,0.275660,0.201643,0.026000
3066,0.247000,0.249727,0.501590,0.474000,0.532580,0.301251,0.026000
3504,0.244600,0.249405,0.501590,0.474000,0.532580,0.301251,0.026000
3942,0.236000,0.250217,0.501590,0.474000,0.532580,0.301251,0.026000
4380,0.226600,0.255480,0.470240,0.549000,0.411240,0.212099,0.100000


#### Save Model Config

In [ ]:
def write_json(path,data,test=False):
  if not isinstance(data, dict):
    data = data.to_dict("records")

  with open(path, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

In [ ]:
summary = {
    "checkpoint" : checkpoint,
    "best_model" : best_model,
    "best_treshold": best_threshold,
    "best_learning_rate": best_learning_rate,
    "empty_labels": empty_labels,
    "train_files" : dataset_memes_2024_files
}

summary

In [ ]:
summary_file_path = summary_dir_path + "summary.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)
write_json(summary_file_path, summary)

Delete output and tmp files

In [ ]:
def delete_dir(path):
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"The directory {path} and all its contents have been deleted successfully")
        except OSError as e:
            print(f"Error: {e.strerror}")
    else:
        print(f"The directory {path} does not exist")

In [ ]:
def delete_output_dirs(parent_directory):
    for entry in os.listdir(parent_directory):
        entry_path = os.path.join(parent_directory, entry)

        if os.path.isdir(entry_path) and entry.startswith('output_'):
            delete_dir(entry_path)

In [ ]:
path = folder_name + "subtask1"
delete_output_dirs(path)
path = path + "/transformer/training/"
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")

## Evaluation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def tokenize(examples):
    encoding = tokenizer(examples["text"], truncation=True,padding=True, max_length=512)
    return encoding

In [ ]:
def get_preds(binarizer,predicted_logits,threshold=0.5):

  predicted_logits_tensor = torch.from_numpy(predicted_logits)
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(predicted_logits_tensor)
  preds=(probs>threshold).int()

  mask = (preds == 1)

  # Use the mask to select the corresponding probabilities
  selected_probs_list = []

  # Iterate over rows and select probabilities using the mask
  for i in range(preds.size(0)):
      selected_probs_row = probs[i][mask[i]].tolist()
      selected_probs_list.append(selected_probs_row)

  names=binarizer.inverse_transform(preds)
  res=[(list(x), y) for x, y in zip(names, selected_probs_list)]
  return res

In [ ]:
project_name="subtask1_multilabel_transformer_encoder_classification"

def return_trainer(model_name,unique_labels):
  api = wandb.Api()
  artifact=api.artifact(model_name)
  model_dir=artifact.download()
  model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]), problem_type="multi_label_classification", ignore_mismatched_sizes=True)

  trainer = Trainer(model=model)
  if torch.cuda.is_available():
    trainer.model = model.cuda()
  return trainer

### Evaluate validation set

In [ ]:
prediction_set=dataset_val['val']
prediction_set=prediction_set.remove_columns(["labels"])

In [ ]:
trainer=return_trainer(best_model, techniques)
threshold=best_threshold
prediction_set_tokenized=prediction_set.map(tokenize, batched=True)

preds=get_preds(mlb,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds=dict(zip(prediction_set_tokenized["id"],preds))

### Evaluate using the scorer script

In [ ]:
def hierarchical_scores(gold_label_path, pred_label_path):
    scorer = folder_name + "subtask1/subtask_1_2a.py"
    command = f'python3 {scorer} --gold_file_path {gold_label_path} --pred_file_path {pred_label_path}'

    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True)
    output = result.stdout.strip()

    parts = output.split('\t')
    f1_h = parts[0].split('=')[1]
    prec_h = parts[1].split('=')[1]
    rec_h = parts[2].split('=')[1]

    f1_h = float(f1_h)
    prec_h = float(prec_h)
    rec_h = float(rec_h)

    hierarchical_metrics = {"f1_hierarchical": f1_h, "precision_hierarchical": prec_h, "recall_hierarchical": rec_h}
    return hierarchical_metrics

In [ ]:
top_k=0
hierarchical_metrics = {"f1_hierarchical": 0, "precision_hierarchical": 0, "recall_hierarchical": 0}
final_df = dict()
for curr_top_k in range(11):
    curr_final_df=dict()
    for k,v in final_ds.items():
      if len(v)>0:
        pred_prob=list(zip(v[0],v[1]))
        pred_prob.sort(key=lambda x: -x[1])

        tmp=[]
        i=0
        for el in pred_prob:
          if i==curr_top_k:
            break
          if el[0] not in tmp:
            tmp.append(el[0])
            i+=1
        curr_final_df[str(k)]=tmp
      else:
        curr_final_df[str(k)]=v

    curr_pred_df=pd.DataFrame.from_dict({"id":curr_final_df.keys(),"labels":curr_final_df.values()})
    curr_val_pred_file="tmp/" + "val_pred.json"
    if not os.path.exists("tmp/"):
        os.makedirs("tmp/")
    write_json(curr_val_pred_file,curr_pred_df)

    curr_hierarchical_metrics = hierarchical_scores(val_path,curr_val_pred_file)

    shutil.rmtree("tmp/")

    print(f"{curr_top_k}:")
    print(curr_hierarchical_metrics)
    if curr_hierarchical_metrics['f1_hierarchical'] >= hierarchical_metrics['f1_hierarchical']:
        hierarchical_metrics = curr_hierarchical_metrics
        top_k = curr_top_k
        final_df = curr_final_df

print(f"\nBest result with top_k = {top_k}")

In [ ]:
pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

val_pred_file=summary_dir_path + "val_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(val_pred_file,pred_df)

hierarchical_scores(val_path,val_pred_file)

In [ ]:
summary["top_k"] = top_k
summary["Validation score"] = hierarchical_metrics

write_json(summary_file_path, summary)

### Create dev output file

In [ ]:
prediction_set=dataset_test["test"]

In [ ]:
trainer=return_trainer(best_model, techniques)
threshold=best_threshold
prediction_set_tokenized=prediction_set.map(tokenize, batched=True)

preds=get_preds(mlb,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds=dict(zip(prediction_set_tokenized["id"],preds))


In [ ]:
final_df=dict()
for k,v in final_ds.items():
  if len(v)>0:
    pred_prob=list(zip(v[0],v[1]))
    pred_prob.sort(key=lambda x: -x[1])

    tmp=[]
    i=0
    for el in pred_prob:
      if i==top_k:
        break
      if el[0] not in tmp:
        tmp.append(el[0])
        i+=1
    final_df[str(k)]=tmp
  else:
    final_df[str(k)]=v


In [ ]:
pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

dev_pred_file=summary_dir_path + "dev_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(dev_pred_file,pred_df)

Delete output and tmp files

In [ ]:
path = folder_name + "subtask1/"
delete_output_dirs(path)
path = path + "/transformer/training/"
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")

In [ ]:
delete_dir('/content/wandb')
delete_dir('/content/tumnlp')
# empty trash of drive
#drive_service.files().emptyTrash().execute()